In [ ]:
!source ~/.zshrc

In [6]:
import os
from sodapy import Socrata
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import rtree
import geopandas as gpd
from shapely.geometry import Point, Polygon
from shapely import wkt
from zipfile import ZipFile
import glob
from datetime import datetime, date


# Establish SodaPy connection
# Establish env variables
username = os.getenv('soc_user')
password = os.getenv('soc_pass')
token = os.getenv('soc_app_token')

# Authenticate client (needed for non-public datasets):
client = Socrata("data.cityofchicago.org", 
                 token, 
                 username=username, 
                 password=password,
                timeout=25)


In [7]:
# Establish zips
q2_zip = ZipFile('data/divvy trips/Divvy_Trips_2018_Q2.zip', 'r')
q3_zip = ZipFile('data/divvy trips/Divvy_Trips_2018_Q3.zip', 'r')
q4_zip = ZipFile('data/divvy trips/Divvy_Trips_2018_Q4.zip', 'r')


# Read in Trip Data
q2 = q2_zip.open('Divvy_Trips_2018_Q2.csv')
q3 = q3_zip.open('Divvy_Trips_2018_Q3.csv')
q4 = q4_zip.open('Divvy_Trips_2018_Q4.csv')

# Create DF
q2_trips = pd.read_csv(q2)
q3_trips = pd.read_csv(q3)
q4_trips = pd.read_csv(q4)


In [8]:
# Bring in Divvy Stations Data
divvy_stations = pd.read_csv('data/divvy_stations.csv')
divvy_stations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 581 entries, 0 to 580
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   station_name      581 non-null    object 
 1   Wards             581 non-null    int64  
 2   total_docks       581 non-null    int64  
 3   latitude          581 non-null    float64
 4   longitude         581 non-null    float64
 5   status            581 non-null    object 
 6   Community Areas   581 non-null    int64  
 7   docks_in_service  581 non-null    int64  
 8   id                581 non-null    int64  
 9   geometry          581 non-null    object 
 10  community num     581 non-null    int64  
 11  side              581 non-null    object 
dtypes: float64(2), int64(6), object(4)
memory usage: 54.6+ KB


In [9]:
# Define Station Look Up Functions
def station_comm_num(id):
    dex = divvy_stations[divvy_stations['id']==id].index
    y = divvy_stations.loc[dex]['community num'].values
    num = str(y).replace("['","").replace("']", "")
    return num

def station_side(id):
    dex = divvy_stations[divvy_stations['id']==id].index
    y = divvy_stations.loc[dex]['side'].values
    side = str(y).replace("['","").replace("']", "")
    return side

In [11]:
group_trips = [q2_trips, q3_trips, q4_trips]
trips_df = pd.concat(group_trips, ignore_index=True)
trips_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3215937 entries, 0 to 3215936
Data columns (total 12 columns):
 #   Column             Dtype  
---  ------             -----  
 0   trip_id            int64  
 1   start_time         object 
 2   end_time           object 
 3   bikeid             int64  
 4   tripduration       object 
 5   from_station_id    int64  
 6   from_station_name  object 
 7   to_station_id      int64  
 8   to_station_name    object 
 9   usertype           object 
 10  gender             object 
 11  birthyear          float64
dtypes: float64(1), int64(4), object(7)
memory usage: 294.4+ MB


In [13]:
# Add Calculations and fix datetimes
trips_df['tripduration'] = trips_df['tripduration'].apply(lambda x: round(float(x.replace(",","")),2))
trips_df['trip duration (min)'] = trips_df['tripduration']/60
trips_df['start_time'] = trips_df['start_time'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
trips_df['end_time'] = trips_df['end_time'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
trips_df['month'] = trips_df['start_time'].apply(lambda x: x.strftime('%b-%y'))

In [23]:
#trips_df['from comm num'] = trips_df['from_station_id'].apply(lambda x: station_comm_num(x))
#trips_df['to comm num'] = trips_df['to_station_id'].apply(lambda x: station_comm_num(x))
#trips_df['from side'] = trips_df['from_station_id'].apply(lambda x: station_side(x))
trips_df['to side'] = trips_df['to_station_id'].apply(lambda x: station_side(x))

In [22]:
# General Aggregates
from_agg_by_side = trips_df.groupby('from side')['trip_id'].agg('count')
from_agg_by_side

from side
[]                  22351
far north side     182714
far south side       1294
loop              1349062
north side         758404
northwest side      10878
south side         152494
southwest side       4380
west side          734360
Name: trip_id, dtype: int64

In [25]:
trips_df.to_csv('data/2018_trips.csv.gz',
               index=False,
                chunksize=100000,
                compression='gzip',
                encoding='utf-8'
               )